In [214]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [215]:
data_dir = 'data/Training Data'
users = ['User001', 'User002', 'User003', 'User004', 'User005', 'User006', 'User007', 'User008', 'User009', 'User0010', 'User0011', 'User0012', 'User0013', 'User0014', 'User0015', 'User0016', 'User0017']

In [216]:
def gyro_activity(gyro_data,threshold):
    gyro_activity=['Standing']
    
    for i in range(1,len(gyro_data)):
        mag = gyro_data.loc[i, 'magnitude']
        if mag >= threshold:
            activity = 'Walking'
        elif gyro_data.loc[i-1, 'magnitude'] < gyro_data.loc[i, 'magnitude'] and mag >= threshold:
            activity = 'Ascending'
        elif gyro_data.loc[i-1, 'magnitude'] > gyro_data.loc[i, 'magnitude'] and mag >= threshold:
            activity = 'Descending'
        else:
            activity = 'Standing'
        gyro_activity.append(activity)
    return gyro_activity


In [217]:
accel_main=pd.DataFrame()
gyro_main=pd.DataFrame()
lin_accel_main=pd.DataFrame()

for user in users:
    accel_data_path = os.path.join(data_dir, user, 'HTC - Front', 'accelDataM.txt')
    gyro_data_path = os.path.join(data_dir, user, 'HTC - Front', 'gyroDataM.txt')
    lin_accel_data_path = os.path.join(data_dir, user, 'HTC - Front', 'linearAccelDataM.txt')

    accel_data = pd.read_csv(accel_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
    gyro_data = pd.read_csv(gyro_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
    lin_accel_data = pd.read_csv(lin_accel_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
   
    accel_data['user']=user
    gyro_data['user']=user
    lin_accel_data['user']=user
    
    accel_data['magnitude']=np.sqrt(accel_data['x']**2 + accel_data['y']**2 + accel_data['z']**2)
    accel_data['mean_mag'] = accel_data['magnitude'].rolling(window=20, min_periods=1).mean()
    
    gyro_data['magnitude'] = np.sqrt(gyro_data['x'] ** 2 + gyro_data['y'] ** 2 + gyro_data['z'] ** 2)
    
    ACCEL_THRESHOLD=accel_data['magnitude'].mean()
    GYRO_THRESHOLD=gyro_data['magnitude'].mean()
    STEP_LENGTH=0.7 #Meters (2.2 Feet)
    
    accel_data['peak'] = accel_data['mean_mag'] > ACCEL_THRESHOLD
    accel_data['step'] = (accel_data['peak'] != accel_data['peak'].shift()) & (accel_data['peak'] == True)
    accel_data['acc_activity']=accel_data['step'].apply(lambda x: 'Moving' if x==True else 'Standing')
    
    gyro_data['activity']=gyro_activity(gyro_data,GYRO_THRESHOLD)

    accel_frames=[accel_main, accel_data]
    gyro_frames=[gyro_main, gyro_data]
    lin_accel_frames=[lin_accel_main, lin_accel_data]
    
    accel_main=pd.concat(accel_frames)
    gyro_main=pd.concat(gyro_frames)
    lin_accel_main=pd.concat(lin_accel_frames)
    

In [218]:
accel_main.shape

(98258, 10)

In [219]:
accel_main.columns

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude', 'mean_mag', 'peak',
       'step', 'acc_activity'],
      dtype='object')

In [220]:
gyro_main.shape

(98262, 7)

In [221]:
gyro_main.columns

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude', 'activity'], dtype='object')

In [222]:
accel_main.groupby(by="acc_activity")["magnitude"].count()

acc_activity
Moving       3427
Standing    94831
Name: magnitude, dtype: int64

In [223]:
gyro_main.groupby(by="activity")["magnitude"].count()

activity
Standing    55481
Walking     42781
Name: magnitude, dtype: int64

In [224]:
accel_main.groupby(by="user")["step"].sum()

user
User001     164
User0010    132
User0011    190
User0012    202
User0013    243
User0014    187
User0015    210
User0016    221
User0017    196
User002     170
User003     170
User004     219
User005     212
User006     227
User007     183
User008     248
User009     253
Name: step, dtype: int64

In [225]:
accel_main.describe()

,x,y,z,magnitude,mean_mag
count,98258.000000,98258.000000,98258.000000,98258.000000,98258.000000
mean,0.609102,-6.365657,0.728163,10.709619,10.709626
std,3.415661,7.620364,4.439676,4.021635,1.279846
min,-22.039999,-39.489998,-38.379997,0.114455,6.508733
25%,-1.310000,-10.639999,-1.510000,8.839416,9.825935
50%,0.410000,-8.760000,0.300000,9.968004,10.395509
75%,2.400000,-3.760000,2.390000,11.960252,11.386998
max,29.670000,38.950000,40.059998,58.095557,18.986161


In [226]:
gyro_main.describe()

,x,y,z,magnitude
count,98262.000000,98262.000000,98262.000000,98262.000000
mean,0.002586,0.032904,-0.010841,1.651970
std,1.307661,1.303422,0.833543,1.172967
min,-6.550000,-11.840000,-7.340000,0.000000
25%,-0.630000,-0.620000,-0.390000,0.805481
50%,0.050000,0.030000,0.000000,1.404457
75%,0.770000,0.680000,0.410000,2.232689
max,7.690000,15.719999,8.040000,15.804973


In [227]:
test_dir = 'data/Testing Data'
test_users = ['User001', 'User002', 'User003', 'User004', 'User005', 'User006', 'User007', 'User008', 'User009', 'User0010', 'User0011', 'User0012', 'User0013', 'User0014', 'User0015', 'User0016', 'User0017']

test_accel_main=pd.DataFrame()
test_gyro_main=pd.DataFrame()
test_lin_accel_main=pd.DataFrame()
test_magnet_main=pd.DataFrame()

for user in test_users:
    test_accel_data_path = os.path.join(test_dir, user, 'HTC - Front', 'accelDataM.txt')
    test_gyro_data_path = os.path.join(test_dir, user, 'HTC - Front', 'gyroDataM.txt')
    test_lin_accel_data_path = os.path.join(test_dir, user, 'HTC - Front', 'linearAccelDataM.txt')
    
    test_accel_data = pd.read_csv(test_accel_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
    test_gyro_data = pd.read_csv(test_gyro_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
    test_lin_accel_data = pd.read_csv(test_lin_accel_data_path, header=None, names=['timestamp', 'x', 'y', 'z'])
    
    test_accel_data['user']=user
    test_gyro_data['user']=user
    test_lin_accel_data['user']=user

    test_accel_frames=[test_accel_main, test_accel_data]
    test_gyro_frames=[test_gyro_main, test_gyro_data]
    test_lin_accel_frames=[test_lin_accel_main, test_lin_accel_data]
    
    test_accel_main=pd.concat(test_accel_frames)
    test_gyro_main=pd.concat(test_gyro_frames)
    test_lin_accel_main=pd.concat(test_lin_accel_frames)

    test_accel_main['magnitude'] = np.sqrt(test_accel_main['x']**2 + test_accel_main['y']**2 + test_accel_main['z']**2)
    test_gyro_main['magnitude'] = np.sqrt(test_gyro_main['x']**2 + test_gyro_main['y']**2 + test_gyro_main['z']**2)


In [228]:
print(test_accel_main.head())
print(test_gyro_main.head())


                 timestamp     x     y         z     user  magnitude
0  2017-04-26 17:01:45:525 -0.28  4.08  8.950000  User001   9.840086
1  2017-04-26 17:01:45:564  0.72  3.56  8.679999  User001   9.409271
2  2017-04-26 17:01:45:606  0.47  3.22  8.900000  User001   9.476249
3  2017-04-26 17:01:45:608 -0.46  2.65  9.650000  User001  10.017814
4  2017-04-26 17:01:45:608 -0.05  2.93  9.420000  User001   9.865283
                 timestamp     x     y     z     user  magnitude
0  2017-04-26 17:01:45:525  0.00  0.00  0.00  User001   0.000000
1  2017-04-26 17:01:45:566 -0.99 -0.75  0.02  User001   1.242175
2  2017-04-26 17:01:45:607 -0.99 -0.53 -0.03  User001   1.123343
3  2017-04-26 17:01:45:609 -0.86 -0.34 -0.03  User001   0.925257
4  2017-04-26 17:01:45:609 -0.94 -0.28 -0.08  User001   0.984073


In [229]:
test_accel_main.shape

(96378, 6)

In [230]:
accel_main.columns

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude', 'mean_mag', 'peak',
       'step', 'acc_activity'],
      dtype='object')

In [231]:
test_accel_main.columns

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude'], dtype='object')

In [232]:
test_gyro_main.shape

(96376, 6)

In [233]:
test_gyro_main.columns

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude'], dtype='object')

In [234]:
print(accel_data.columns)

Index(['timestamp', 'x', 'y', 'z', 'user', 'magnitude', 'mean_mag', 'peak',
       'step', 'acc_activity'],
      dtype='object')


In [235]:
accel_data = accel_data.drop(['mean_mag', 'peak', 'step'], axis=1)

X_train = accel_data.drop('acc_activity', axis=1).to_numpy()
X_test = test_accel_data.to_numpy()

y_train = accel_data['acc_activity'].to_numpy()
y_test = test_accel_data.to_numpy()




In [ ]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model with a loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model using your training data
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
